# tshark

In [11]:
!tshark autostop packets:20


Running as user "root" and group "root". This could be dangerous.
Capturing on 'wlp2s0'
tshark: Invalid capture filter "autostop packets:20" for interface 'wlp2s0'.

That string isn't a valid capture filter (can't parse filter expression: syntax error).
See the User's Guide for a description of the capture filter syntax.
0 packets captured


# sharkd

# Scapy

### Install

In [3]:
!pip3 install scapy

## Basics

In [4]:
from scapy.all import IP

# create a basic IP-Packet
IP().show()

###[ IP ]### 
  version   = 4
  ihl       = None
  tos       = 0x0
  len       = None
  id        = 1
  flags     = 
  frag      = 0
  ttl       = 64
  proto     = hopopt
  chksum    = None
  src       = 127.0.0.1
  dst       = 127.0.0.1
  \options   \



### Monitoring
#### a simple network scanner

In [12]:
!ip address

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host 
       valid_lft forever preferred_lft forever
2: wlp2s0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc fq_codel state UP group default qlen 1000
    link/ether c4:8e:8f:f3:9b:a7 brd ff:ff:ff:ff:ff:ff
    inet 192.168.178.22/24 brd 192.168.178.255 scope global dynamic noprefixroute wlp2s0
       valid_lft 860978sec preferred_lft 860978sec
    inet6 2001:4dd3:8f2a:0:8c55:12e8:cf1a:ffbb/64 scope global temporary dynamic 
       valid_lft 6937sec preferred_lft 3337sec
    inet6 2001:4dd3:8f2a:0:2a1d:3830:67ba:c18e/64 scope global dynamic mngtmpaddr noprefixroute 
       valid_lft 6937sec preferred_lft 3337sec
    inet6 fe80::470:6b4c:4354:bc76/64 scope link noprefixroute 
       valid_lft forever preferred_lft forever


In [5]:
# Are you root?
# Make sure you have elevated privileges for the next step
!whoami

root


In [6]:
# src: https://www.thepythoncode.com/article/building-networks-scanner-using-scapy
from scapy.all import ARP, Ether, srp
# set target IP range
target = "192.168.178.1/24"
# create ARP (Address Resolution protocol) packet
arp = ARP(pdst=target)
ether = Ether(dst="ff:ff:ff:ff:ff:ff")
packet = ether/arp

# raw socket operations require elevated privileges
result = srp(packet, timeout=10) # => Tuple (answered, unanswered)

print("===== Query Answer =====")
print(result[0][0])

print("===== Devices =====")
{received.psrc: received.hwsrc for sent, received in result[0]}

Begin emission:
Finished sending 256 packets.
*******..............
Received 21 packets, got 7 answers, remaining 249 packets
===== Query Answer =====
QueryAnswer(query=<Ether  dst=ff:ff:ff:ff:ff:ff type=ARP |<ARP  pdst=192.168.178.1 |>>, answer=<Ether  dst=c4:8e:8f:f3:9b:a7 src=3c:37:12:3b:b4:a6 type=ARP |<ARP  hwtype=0x1 ptype=IPv4 hwlen=6 plen=4 op=is-at hwsrc=3c:37:12:3b:b4:a6 psrc=192.168.178.1 hwdst=c4:8e:8f:f3:9b:a7 pdst=192.168.178.22 |>>)
===== Devices =====


{'192.168.178.1': '3c:37:12:3b:b4:a6',
 '192.168.178.20': '60:45:cb:9a:3a:bd',
 '192.168.178.29': '00:f6:20:80:9c:75',
 '192.168.178.38': '34:ab:95:1c:00:d3',
 '192.168.178.39': 'c4:5b:be:e4:47:eb',
 '192.168.178.40': '18:69:d8:c9:1d:0e',
 '192.168.178.36': 'c8:c9:a3:cb:34:54'}

#### a simple network sniffer

In [ ]:
# adapted from src: https://www.thepythoncode.com/article/dhcp-listener-using-scapy-in-python
from scapy.all import sniff, Ether, DHCP
import time

global print_packet, target_mac, requested_ip, hostname, vendor_id

def listen():
    sniff(prn=print_packet, iface='wlp2s0')

def print_packet(packet):
    print(type(packet), packet)

listen()